# Layer plot

In [1]:
import plotly.express as px
import plotly.graph_objects as go

import pandas as pd 
import wandb
api = wandb.Api()

# Project is specified by <entity/project-name>
runs = api.runs("nils2/dl4nlp")

summary_list, config_list, name_list = [], [], []
for run in runs: 
    if run.state != "finished":
        continue
    # .summary contains the output keys/values for metrics like accuracy.
    #  We call ._json_dict to omit large files 
    summary_list.append(run.summary._json_dict)

    # .config contains the hyperparameters.
    #  We remove special values that start with _.
    config_list.append(
        {k: v for k,v in run.config.items()
          if not k.startswith('_')})

    # .name is the human-readable name of the run.
    name_list.append(run.name)

runs_df = pd.DataFrame({
    "summary": summary_list,
    "config": config_list,
    "name": name_list
    })

runs_df.to_csv("project.csv")

In [2]:
mbert_uni = runs_df[runs_df.name.str.startswith("mbert-bbs32-unic-l")]
xlmbert_uni = runs_df[runs_df.name.str.startswith("xlmbert-bbs32-unic-l")]
mbert = runs_df[runs_df.name.str.startswith("mbert-bbs32-l")]
xlmbert = runs_df[runs_df.name.str.startswith("xlmbert-bbs32-l")]

In [3]:
mbert_uni = pd.concat([mbert_uni,runs_df[runs_df.name.str.fullmatch("mbert-bbs32-unic")]])
xlmbert_uni = pd.concat([runs_df[runs_df.name.str.fullmatch("xlmbert-bbs32-unic")],xlmbert_uni])
mbert = pd.concat([mbert,runs_df[runs_df.name.str.fullmatch("mbert-bbs32")]])
xlmbert = pd.concat([xlmbert,runs_df[runs_df.name.str.fullmatch("xlmbert-bbs32-adam")]])

In [4]:
mbertun_res = pd.DataFrame({
        "test/accuracy": [s["test/accuracy"] for s in mbert_uni.summary],
        "embeddings_layer": [c["embeddings_layer"] for c in mbert_uni.config],
})
xlmbertun_res = pd.DataFrame({
        "test/accuracy": [s["test/accuracy"] for s in xlmbert_uni.summary],
        "embeddings_layer": [c["embeddings_layer"] for c in xlmbert_uni.config],
})
mbert_res = pd.DataFrame({
        "test/accuracy": [s["test/accuracy"] for s in mbert.summary],
        "embeddings_layer": [c["embeddings_layer"] for c in mbert.config],
})
xlmbert_res = pd.DataFrame({
        "test/accuracy": [s["test/accuracy"] for s in xlmbert.summary],
        "embeddings_layer": [c["embeddings_layer"] for c in xlmbert.config],
})

In [5]:
mbertun_res = mbertun_res.sort_values(by="embeddings_layer")
xlmbertun_res = xlmbertun_res.sort_values(by="embeddings_layer")
mbert_res = mbert_res.sort_values(by="embeddings_layer")
xlmbert_res = xlmbert_res.sort_values(by="embeddings_layer")

In [7]:
fig = go.Figure(layout=dict(height=700, width=900))
fig.add_trace(go.Scatter(x=mbertun_res["embeddings_layer"], y=mbertun_res["test/accuracy"],
                    mode='lines',
                    line=dict(dash='dot', color='Red'),
                    name='MBERT-unicode-norm'))
fig.add_trace(go.Scatter(x=xlmbertun_res["embeddings_layer"], y=xlmbertun_res["test/accuracy"],
                         mode='lines',
                    line=dict(dash='dot', color='Green'),
                    name='XLMRoberta-unicode-norm'))
fig.add_trace(go.Scatter(x=mbert_res["embeddings_layer"], y=mbert_res["test/accuracy"],
                    mode='lines',
                    line=dict(color='Orange'),
                    name='MBERT'))
fig.add_trace(go.Scatter(x=xlmbert_res["embeddings_layer"], y=xlmbert_res["test/accuracy"],
                         mode='lines',
                    line=dict(color='Cyan'),
                    name='XLMRoberta'))
fig.update_layout(plot_bgcolor='White',
                  xaxis_title="Layer",
                  yaxis_title='Test accuracy')
fig.update_xaxes(showgrid=True, gridwidth=1, linecolor='Black', gridcolor='LightGrey')
fig.update_yaxes(showgrid=True, gridwidth=1, linecolor='Black', gridcolor='LightGrey')
fig.show()

# Parameter plots

## Probe batch size

In [ ]:
mbert_bs64 = runs_df[runs_df.name.str.fullmatch("mbert-bbs32-bs64")]
mbert_bs128 = runs_df[runs_df.name.str.fullmatch("mbert-bbs32-bs128")]
mbert_bs256 = runs_df[runs_df.name.str.fullmatch("mbert-bbs32-bs256")]
mbert_bs512 = runs_df[runs_df.name.str.fullmatch("mbert-bbs32-bs512")]
mbert_bs1024 = runs_df[runs_df.name.str.fullmatch("mbert-bbs32-bs1024")]

mbert_bs = pd.concat([mbert_bs64, mbert_bs128, mbert_bs256, mbert_bs512, mbert_bs1024])

In [ ]:
mbert_bs = pd.DataFrame({
        "test/accuracy": [s["test/accuracy"] for s in mbert_bs.summary],
        "bs": [c["batch_size"] for c in mbert_bs.config],
})

In [ ]:
fig = go.Figure(layout=dict(height=700, width=900))
fig.add_trace(go.Bar(x=mbert_bs["bs"], y=mbert_bs["test/accuracy"],
                     marker=dict(color = mbert_bs["bs"],
                     colorscale='rainbow')))
fig.update_layout(xaxis_title="Probe batch size",
                   yaxis_title='Test accuracy',
                 xaxis=dict(type='category'),
                 yaxis_range=[0.85,0.95])
fig.show()

## Embeddings batch size

In [ ]:
mbert_bs16 = runs_df[runs_df.name.str.fullmatch("mbert-bbs16")]
mbert_bs32 = runs_df[runs_df.name.str.fullmatch("mbert-bbs32")]
mbert_bs64 = runs_df[runs_df.name.str.fullmatch("mbert-bbs64")]

mbert_bs = pd.concat([mbert_bs16, mbert_bs32, mbert_bs64])

In [ ]:
mbert_bs = pd.DataFrame({
        "test/accuracy": [s["test/accuracy"] for s in mbert_bs.summary],
        "bs": [c["bert_batch_size"] for c in mbert_bs.config],
})

In [ ]:
fig = go.Figure(layout=dict(height=700, width=900))
fig.add_trace(go.Bar(x=mbert_bs["bs"], y=mbert_bs["test/accuracy"],
                     marker=dict(color = mbert_bs["bs"],
                     colorscale='rainbow')))
fig.update_layout(xaxis_title="Embedding batch size",
                   yaxis_title='Test accuracy',
                 xaxis=dict(type='category'),
                 yaxis_range=[0.85,0.95])
fig.show()

## Activations

In [ ]:
mbert_bsT = runs_df[runs_df.name.str.fullmatch("mbert-bbs32-tanh")]
mbert_bsR = runs_df[runs_df.name.str.fullmatch("mbert-bbs32")]
mbert_bsS = runs_df[runs_df.name.str.fullmatch("mbert-bbs32-sigmoid")]

mbert_bs = pd.concat([mbert_bsT, mbert_bsR, mbert_bsS])

In [ ]:
mbert_bs = pd.DataFrame({
        "test/accuracy": [s["test/accuracy"] for s in mbert_bs.summary],
        "act": [c["activation"] for c in mbert_bs.config],
})

In [ ]:
fig = go.Figure(layout=dict(height=700, width=900))
fig.add_trace(go.Bar(x=mbert_bs["act"], y=mbert_bs["test/accuracy"]))
fig.update_layout(xaxis_title="Activation function",
                   yaxis_title='Test accuracy',
                 xaxis=dict(type='category'),
                 yaxis_range=[0.85,0.95])
fig.show()

## Optimizer

In [ ]:
mbert_bsa = runs_df[runs_df.name.str.fullmatch("mbert-bbs32-adam")]
mbert_bs = runs_df[runs_df.name.str.fullmatch("mbert-bbs32")]
mbert_bsw = runs_df[runs_df.name.str.fullmatch("mbert-bbs32-adamw")]

mbert_bs = pd.concat([mbert_bs, mbert_bsa, mbert_bsw])

In [ ]:
mbert_bs = pd.DataFrame({
        "test/accuracy": [s["test/accuracy"] for s in mbert_bs.summary],
        "opt": [c["optimizer"] for c in mbert_bs.config],
})

In [ ]:
fig = go.Figure(layout=dict(height=700, width=900))
fig.add_trace(go.Bar(x=mbert_bs["opt"], y=mbert_bs["test/accuracy"]))
fig.update_layout(xaxis_title="Optimizer function",
                   yaxis_title='Test accuracy',
                 xaxis=dict(type='category'),
                 yaxis_range=[0.75,0.95])
fig.show()

### SGD Momentum 

In [ ]:
mbert_bs0 = runs_df[runs_df.name.str.fullmatch("mbert-bbs32-m08")]
mbert_bs = runs_df[runs_df.name.str.fullmatch("mbert-bbs32")]
mbert_bs5 = runs_df[runs_df.name.str.fullmatch("mbert-bbs32-m095")]

mbert_bs = pd.concat([mbert_bs0, mbert_bs, mbert_bs5])

In [ ]:
mbert_bs = pd.DataFrame({
        "test/accuracy": [s["test/accuracy"] for s in mbert_bs.summary],
        "m": [c["momentum"] for c in mbert_bs.config],
})

In [ ]:
fig = go.Figure(layout=dict(height=700, width=900))
fig.add_trace(go.Bar(x=mbert_bs["m"], y=mbert_bs["test/accuracy"]))
fig.update_layout(xaxis_title="SGD momentum rates",
                   yaxis_title='Test accuracy',
                 xaxis=dict(type='category'),
                 yaxis_range=[0.9,0.93])
fig.show()

### SGD Learning rate

In [ ]:
mbert_bs0 = runs_df[runs_df.name.str.fullmatch("mbert-lr005-bbs32")]
mbert_bs = runs_df[runs_df.name.str.fullmatch("mbert-bbs32")]
mbert_bs5 = runs_df[runs_df.name.str.fullmatch("mbert-lr05-bbs32")]

mbert_bs = pd.concat([mbert_bs0, mbert_bs, mbert_bs5])

In [ ]:
mbert_bs = pd.DataFrame({
        "test/accuracy": [s["test/accuracy"] for s in mbert_bs.summary],
        "lr": [c["learning_rate"] for c in mbert_bs.config],
})

In [ ]:
fig = go.Figure(layout=dict(height=700, width=900))
fig.add_trace(go.Bar(x=mbert_bs["lr"], y=mbert_bs["test/accuracy"]))
fig.update_layout(xaxis_title="SGD learning rates",
                   yaxis_title='Test accuracy',
                 xaxis=dict(type='category'),
                 yaxis_range=[0.9,0.93])
fig.show()